In [2]:
from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate(
    [
        ("system","you are helpful {domain} expert"),
        ("human","Explain the given {topic} in simple terms")
    ]
)


In [6]:
prompt = chat_template.invoke({"domain":"medical","topic":"cancer"})

In [7]:
print(prompt)

messages=[SystemMessage(content='you are helpful medical expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the given cancer in simple terms', additional_kwargs={}, response_metadata={})]


In [9]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
groq_model=ChatGroq(model="deepseek-r1-distill-llama-70b")

In [11]:
template = PromptTemplate(
    template="generate summary for given text {text}"
    ,input_variables=['text']
)

## Json Output Parser

In [34]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_parser.get_format_instructions()

'Return a JSON object.'

In [35]:
template=PromptTemplate(
    template="give me name, age and city from the provided text  {text} \n {format_instructions}",
    input_variables=['text'],
    partial_variables={"format_instructions":json_parser.get_format_instructions()}
)

In [36]:
text="HI My name is Venky and age is 35 ,staying in Bangalore"
prompt_v1 = template.format(text=text)

In [37]:
result = groq_model.invoke(prompt_v1)
result

AIMessage(content='<think>\nAlright, so the user has given me a query where they want me to extract the name, age, and city from a provided text and return it as a JSON object. Let me break this down.\n\nFirst, I need to understand the structure of the input. The text is: "HI My name is Venky and age is 35 ,staying in Bangalore". It looks like a simple sentence with the necessary information embedded within it.\n\nMy goal is to parse this text and extract the specific pieces of information. The name is clearly stated as "Venky". That seems straightforward. Next, the age is given as 35. I should make sure to represent this as a number in the JSON, not a string. Then, the city is mentioned as Bangalore.\n\nNow, I need to structure this into a JSON object. JSON keys are usually in lowercase and separated by underscores if needed. So, the keys will be "name", "age", and "city". Each will have the corresponding value extracted from the text.\n\nI should also check if there are any commas or

In [38]:
result.content

'<think>\nAlright, so the user has given me a query where they want me to extract the name, age, and city from a provided text and return it as a JSON object. Let me break this down.\n\nFirst, I need to understand the structure of the input. The text is: "HI My name is Venky and age is 35 ,staying in Bangalore". It looks like a simple sentence with the necessary information embedded within it.\n\nMy goal is to parse this text and extract the specific pieces of information. The name is clearly stated as "Venky". That seems straightforward. Next, the age is given as 35. I should make sure to represent this as a number in the JSON, not a string. Then, the city is mentioned as Bangalore.\n\nNow, I need to structure this into a JSON object. JSON keys are usually in lowercase and separated by underscores if needed. So, the keys will be "name", "age", and "city". Each will have the corresponding value extracted from the text.\n\nI should also check if there are any commas or spaces that might

In [27]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

In [39]:
json_parser.parse(result.content)

{'name': 'Venky', 'age': 35, 'city': 'Bangalore'}

In [40]:
chain = template | groq_model | json_parser

In [41]:
chain.invoke({"text":text})

{'name': 'Venky', 'age': 35, 'city': 'Bangalore'}

In [42]:
template2 = PromptTemplate(
    template = "Gimme 5 facts about {topic} \n {format_instructions}",
    input_variables= ['topic'],
    partial_variables={"format_instructions":json_parser.get_format_instructions()}   

)

### Generate Structure Output

In [43]:
from langchain.output_parsers import StructuredOutputParser,ResponseSchema
schema = [
ResponseSchema(name="first_fact",description="first fact about text"),
ResponseSchema(name="second_fact",description="second fact about text"),
ResponseSchema(name="third_fact",description="third fact about text"),
]

### Pydantic Output Parser

In [46]:
from pydantic import BaseModel,Field
from langchain_core.output_parsers import PydanticOutputParser

class Person(BaseModel):
    name:str = Field(description="Name of the person")
    age:int = Field(description="age of the person",gt=18)
    city:str = Field(description="Name of the city where person is located")

In [54]:
py_parser = PydanticOutputParser(pydantic_object= Person)
py_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city where person is located", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'

In [55]:
template=PromptTemplate(
    template="give me name, age and city from the provided text  {text} \n {format_instructions}",
    input_variables=['text'],
    partial_variables={"format_instructions":py_parser.get_format_instructions()}
)

In [57]:
text="HI My name is Venky and age is 35 ,staying in Bangalore"
prompt_v2 = template.format(text=text)
prompt_v2

'give me name, age and city from the provided text  HI My name is Venky and age is 35 ,staying in Bangalore \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city where person is located", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'

In [58]:
groq_model.invoke(prompt_v2)

AIMessage(content='<think>\nOkay, so the user gave me this query where they want me to extract name, age, and city from a provided text and format it as a JSON following a specific schema. Let me break this down.\n\nFirst, I need to understand the input. The text is "HI My name is Venky and age is 35 ,staying in Bangalore". So, I should extract "Venky" as the name, "35" as the age, and "Bangalore" as the city.\n\nLooking at the output schema, it requires a JSON object with properties name, age, and city. Each has specific types: name and city are strings, age is an integer. Also, age must be greater than 18. In the example, 35 is way above 18, so that\'s fine.\n\nI should make sure that the JSON is correctly formatted. So, the structure should be like:\n{\n  "name": "Venky",\n  "age": 35,\n  "city": "Bangalore"\n}\n\nI also need to ensure that the JSON doesn\'t have any extra properties and strictly follows the schema. No additional fields should be present. \n\nWait, in the text, ther